In [3]:
import numpy as np

import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pickle

import time

import os

In [78]:
os.chdir('Data')

In [16]:
def get_soup(url):
    driver = webdriver.Chrome(executable_path='/anaconda3/bin/chromedriver')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

In [18]:
soup = get_soup('https://www.indeed.com/jobs?q=data+scientist&l=New+York')

In [ ]:
for i in list(range(0,1000,10)):
    base_url = 'https://ca.indeed.com/jobs?q={}&l={}&start={}'.format(query, location, i)
    try:
        soup = get_soup(base_url)
        urls += grab_job_links(soup)
    except:
        continue

In [28]:
def grab_job_links(query):
    """Enter a search query and the function will output a list of URLs with the proper page information.
    e.g. if there are 30 jobs there will only be URLs with page numbers going up to 30.
    Were the page number in a search to go higher the last page would simply be returned repeatedly."""
    
    urls = []
    
    num_postings_str = soup.find(name='div', attrs={'id':"searchCount"}).get_text().split()[-2]
    num_postings_str = int(num_postings_str.replace(',',''))
    num_postings_str
    
    for i in list(range(0,1000,10)):
        base_url = 'https://www.indeed.com/jobs?q=data+scientist&l=New+York&start={}'.format(i)
        soup = get_soup(base_url)
    
        for link in soup.find_all('h2', {'class': 'jobtitle'}):
            try:
                partial_url = link.a.get('href')
                url = 'https://ca.indeed.com' + partial_url
                urls.append(url)

            except:
                continue
        
    return urls

In [29]:
urls = grab_job_links('https://www.indeed.com/jobs?q=data+scientist&l=New+York&start={}')

Locations and key words on which to search

In [57]:
queries = [['Data','Scientist'],['Machine','Learning','Engineer'],
             ['Data','Engineer'],['Research','Engineer'],
             ['Artificial','Intelligence','Engineer'],
             ['Product','Analyst'],['Product','Scientist']]

locations = [['Seattle'],['New','York,+NY&_ga='],
             ['Los','Angeles'],['Chicago'],['San','Francisco','Bay','Area%2C+CA']]

Indeed will only return the first 1000 results when.  
Some queries have more jobs but the same jobs will continue to be returned after 1000.

In [ ]:
s_query = '+'.join(queries[0])
s_location = '+'.join(locations[0])
soup_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(s_query, s_location, 0)
print(soup_url)

soup = get_soup(soup_url)

num_postings_str = soup.find(name='div', attrs={'id':"searchCount"}).get_text().split()[-2]
num_postings = int(num_postings_str.replace(',',''))



if num_postings > 1000:
    num_postings = 1000
    
range(0,num_postings,10)

In [68]:
p_query = ''.join(queries[0])
p_location = ''.join(locations[0])

str('urls_to_query_' + '_'.join([p_query,p_location]))

'urls_to_query_DataScientist_Seattle'

Creates list of urls that will be scraped to get actual job URLs.

In [79]:
for location in locations:
    
    urls = []
    
    for query in queries:
        
        time.sleep(np.random.poisson(100)/50)
    
        s_query = '+'.join(query)
        s_location = '+'.join(location)
        soup_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(s_query, s_location, 0)

        soup = get_soup(soup_url)

        num_postings_str = soup.find(name='div', attrs={'id':"searchCount"}).get_text().split()[-2]
        num_postings = int(num_postings_str.replace(',',''))

        if num_postings > 1000:
            num_postings = 1000

        for i in range(0,num_postings,10):
            s_query = '+'.join(query)
            s_location = '+'.join(location)
            base_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(s_query, s_location, i)
            
            urls.append(base_url)
            
    p_location = ''.join(location)
    pickle_name = str('urls_to_query__' + p_location)

    pickle_out = open(pickle_name,'wb')
    pickle.dump(urls, pickle_out)
    pickle_out.close()

In [73]:
time.sleep(np.random.poisson(100)/50)

In [ ]:
base_url = 'https://ca.indeed.com/jobs?q={}&l={}&start={}'.format(query, location, num)

In [32]:
pickle_out = open("NewYork_DataScientist","wb")
pickle.dump(urls, pickle_out)
pickle_out.close()

In [36]:
pickle_in = open('NewYork_DataScientist','rb')
NewYork_DataScientist = pickle.load(pickle_in)
pickle_in.close()

In [24]:
for i in list(range(0,1000,10)):
    base_url = 'https://www.indeed.com/jobs?q=data+scientist&l=New+York&start={}'.format(i)
    try:
        soup = get_soup(base_url)
        urls += grab_job_links(soup)
    except:
        continue